## json_df1: Pulls all JSONS from all metadata patchseq

In [1]:
import os
import csv
import json
import fnmatch
import numpy as np
import pandas as pd
from IPython.display import display #Found this to display all columns in a table
from pandas.io.json import json_normalize
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def get_jsons(dirname, expt):
    json_paths = []
    for jfile in os.listdir(dirname):
        if fnmatch.fnmatch(jfile, '*.%s.json' %expt):
            jpath = os.path.join(dirname, jfile)
            json_paths.append(jpath) 
    return json_paths

In [3]:
#make list of json paths for every json file in MIES Experiments directory on 279
json_list = get_jsons("//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/", "PS")
json_list

['//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176.11.06.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Rbp4-Cre_KL100;Ai14-357010.03.01.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Oxtr-T2A-Cre;Pvalb-T2A-FlpO;Ai65-370683.03.01.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Ntsr1-Cre_GN220;Ai14-361815.05.01.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Gad2-IRES-Cre;Ai14-296621.05.02.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Cux2-CreERT2;Ai14-315357.03.02.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Rbp4-Cre_KL100;Ai14-280355.05.01.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/H17.06.015.13.08.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all

In [4]:
from pandas.io.json import json_normalize

def flatten_attempts(slice_info, form_version):
    """Return flattened slice metadata dataframe.
    
    Parameters
    ----------
    slice_info : dict
        A dictionary of slices with nested pipette attempts.
    
    form_version : string
        A string containing the JEM form version.
        (Pre-version 2 contains IVSCC, PatchSeq and Electroporation arrays)
    Returns
    -------
    attempts_slice_df : pandas dataframe
        A dataframe of all pipette attempts along with all slice metadata.
    """
    
    df = json_normalize(slice_info)
    if form_version >= "2":
        ps_array_name = "pipettes"
    else:
        ps_array_name = "pipettesPatchSeqPilot"
    try:
        attempts_df = json_normalize(slice_info[ps_array_name])
        attempts_df["limsSpecName"] = df["limsSpecName"].values[0]
        attempts_df["attempt"] = [p+1 for p in attempts_df.index.values]
        attempts_slice_df = pd.merge(df, attempts_df, how="outer", on="limsSpecName")
        attempts_slice_df.drop(ps_array_name, axis=1, inplace=True)
        return attempts_slice_df
    except IndexError:
        pass

In [5]:
def is_field(df, colname):
    """Determine whether a column name exists in a dataframe.
    
    Parameters
    ----------
    df : a Pandas dataframe
    colname : string
        
    Returns
    -------
    Boolean
        Boolean value indicating if the colname exists in the dataframe.
    """
    
    try:
        df[colname]
        return True
    except KeyError:
        return False

##### Output Starts Here

In [6]:
json_df = pd.DataFrame()

for json_path in json_list:
    with open(json_path) as data_file:
        slice_info = json.load(data_file)
        if is_field(slice_info, "formVersion"):
            jem_version = slice_info["formVersion"]
        else:
            jem_version = "1.0.0"
        flat_df = flatten_attempts(slice_info, jem_version)
        json_df = pd.concat([json_df, flat_df], axis=0)
        
pd.options.display.max_columns = None
display(json_df.head()) #Displays all the columns horizontally scroll if super long

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,date,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID
0,2018-04-18,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,1.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:20:14 -07:00,13:20:13 -07:00,13:20:14 -07:00,NaN,PXS4_180418_551_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:19:59 -07:00,13:20:00 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
1,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,2.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_present,-80.0,-80.0,NaN,NaN,13:21:44 -07:00,13:21:44 -07:00,13:21:45 -07:00,NaN,PXS4_180418_552_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:21:26 -07:00,13:21:31 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
2,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,3.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:22:39 -07:00,13:22:38 -07:00,13:22:39 -07:00,NaN,PXS4_180418_553_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:22:25 -07:00,13:22:27 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
3,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,4.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,not_intentionally,nucleus_present,-80.0,-80.0,NaN,NaN,13:23:59 -07:00,13:23:58 -07:00,13:23:59 -07:00,NaN,PXS4_180418_554_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:23:04 -07:00,13:23:09 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
4,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,5.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_absent,-80.0,-80.0,NaN,NaN,13:25:01 -07:00,13:25:01 -07:00,13:25:01 -07:00,NaN,PXS4_180418_555_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:24:37 -07:00,13:24:40 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN


#### Filtering columns

In [7]:
json_df = json_df[json_df['approach.creCell'] == 'Cre+'] 
json_df = json_df[json_df['status'] == 'SUCCESS']

In [8]:
display(json_df.head())

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,date,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID
0,2018-04-18,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,1.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:20:14 -07:00,13:20:13 -07:00,13:20:14 -07:00,NaN,PXS4_180418_551_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:19:59 -07:00,13:20:00 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
1,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,2.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_present,-80.0,-80.0,NaN,NaN,13:21:44 -07:00,13:21:44 -07:00,13:21:45 -07:00,NaN,PXS4_180418_552_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:21:26 -07:00,13:21:31 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
2,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,3.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:22:39 -07:00,13:22:38 -07:00,13:22:39 -07:00,NaN,PXS4_180418_553_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:22:25 -07:00,13:22:27 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
3,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,4.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,not_intentionally,nucleus_present,-80.0,-80.0,NaN,NaN,13:23:59 -07:00,13:23:58 -07:00,13:23:59 -07:00,NaN,PXS4_180418_554_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:23:04 -07:00,13:23:09 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
4,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,5.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_absent,-80.0,-80.0,NaN,NaN,13:25:01 -07:00,13:25:01 -07:00,13:25:01 -07:00,NaN,PXS4_180418_555_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:24:37 -07:00,13:24:40 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN


In [9]:
display(json_df.tail())

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,date,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID
0,11/01/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,10/12/2017,11/02/2017 12:03,NaN,NaN,NaN,1000,alexa fill visible post experiment.,Cell Swelled,no,nucleus_present,-100.0,-50.0,No Bubbles,NaN,12:31:55,12:27:45,12:35:06,12:31:55,053,NaN,NaN,Yes,2.0.0,NaN,10/05/2017,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai14-354918.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,12:18:43,12:20:40,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171030_03_B02_M
1,11/01/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 6a",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,10/12/2017,11/02/2017 12:03,NaN,NaN,NaN,1000,alexa fill visible post experiment.,NaN,no,nucleus_absent,-60.0,-25.0,No Bubbles,NaN,13:01:10,12:58:33,13:03:05,13:01:11,054,NaN,NaN,Yes,2.0.0,NaN,10/05/2017,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai14-354918.04.01,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,12:49:54,12:51:25,5,rustym,NaN,NaN,SUCCESS,Patch/Cell Unstable,EPhys_171030_03_B02_M
0,10/25/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,10/12/2017,10/26/2017 14:36,NaN,NaN,NaN,15,alexa fill visible post experiment. Forgot to...,Fluorescence in Pipette,intentionally,nucleus_present,-25.0,-25.0,No Bubbles,NaN,15:05:15,15:04:15,15:08:05,15:05:15,053,NaN,NaN,Yes,2.0.0,NaN,10/05/2017,NaN,NaN,NaN,NaN,NaN,Oxtr-T2A-Cre;Ai14-355298.03.01,NaN,NaN,NaN,NaN,NaN,NaN,6.2,NaN,14:53:08,14:55:01,4,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171023_03_B04_M
1,10/25/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,2.0,NaN,NaN,10/12/2017,10/26/2017 14:36,NaN,NaN,NaN,1000,alexa fill visible post experiment,Fluorescence in Pipette,no,nucleus_present,-25.0,-25.0,Small Bubbles,NaN,15:31:38,15:30:51,15:33:33,15:31:38,054,NaN,NaN,Yes,2.0.0,NaN,10/05/2017,NaN,NaN,NaN,NaN,NaN,Oxtr-T2A-Cre;Ai14-355298.03.01,NaN,NaN,NaN,NaN,NaN,NaN,4.1,NaN,15:21:46,15:23:15,4,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171023_03_B04_M
1,10/02/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 4",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,10/03/2017 14:35,NaN,NaN,NaN,25,Alexa fill not visible,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_absent,-25.0,-25.0,Small Bubbles,NaN,14:58:42,14:58:16,15:03:40,14:58:44,356,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Nr5a1-Cre;Ai14-350880.05.02,NaN,NaN,NaN,NaN,NaN,NaN,4.8,NaN,14:50:27,14:51:26,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_D04_M


#### Sort by recording date

In [25]:
json_df.sort_values('date')

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,date,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID
2,03/20/2018,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,1,3.0,NaN,NaN,02/07/2018,03/20/2018 09:57,NaN,NaN,NaN,6,NaN,NaN,NaN,nucleus_present,0.0,-20.0,NaN,NaN,09:59:32,09:59:32,09:59:33,09:59:32,158,NaN,NaN,No,2.0.0,NaN,03/20/2018,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-382166...,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,09:59:21,09:59:23,MP3,alexh,multipatch patch seq pilot,NaN,SUCCESS,NaN,multi_180319_21_B02
0,03/20/2018,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,02/07/2018,03/20/2018 09:57,NaN,NaN,NaN,6,NaN,NaN,no,nucleus_absent,0.0,-20.0,NaN,NaN,09:58:22,09:58:22,09:58:23,09:58:23,156,NaN,NaN,No,2.0.0,NaN,03/20/2018,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-382166...,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,09:58:10,09:58:12,MP3,alexh,multipatch patch seq pilot,NaN,SUCCESS,NaN,multi_180319_21_B02
1,03/20/2018,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,1,2.0,NaN,NaN,02/07/2018,03/20/2018 09:57,NaN,NaN,NaN,6,NaN,NaN,no,nucleus_present,0.0,-30.0,NaN,NaN,09:59:04,09:59:04,09:59:05,09:59:04,157,NaN,NaN,No,2.0.0,NaN,03/20/2018,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-382166...,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,09:58:49,09:58:51,MP3,alexh,multipatch patch seq pilot,NaN,SUCCESS,NaN,multi_180319_21_B02
0,03/20/2018,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,02/07/2018,03/20/2018 16:34,NaN,NaN,NaN,6,NaN,NaN,no,nucleus_absent,0.0,-30.0,NaN,NaN,09:51:49,09:51:48,09:51:50,09:51:50,151,NaN,NaN,No,2.0.0,NaN,03/20/2018,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-382166...,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,09:50:36,09:50:39,MP3,alexh,multipatch patch seq pilot,NaN,SUCCESS,NaN,multi_180319_21_A04
3,03/20/2018,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,4.0,NaN,NaN,02/07/2018,03/20/2018 16:34,NaN,NaN,NaN,6,NaN,NaN,no,nucleus_absent,0.0,-30.0,NaN,NaN,09:54:27,09:54:27,09:54:28,09:54:28,154,NaN,NaN,No,2.0.0,NaN,03/20/2018,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-382166...,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,09:54:14,09:54:17,MP3,alexh,multipatch patch seq pilot,NaN,SUCCESS,NaN,multi_180319_21_A04
4,03/20/2018,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,5.0,NaN,NaN,02/07/2018,03/20/2018 16:34,NaN,NaN,NaN,6,NaN,NaN,no,nucleus_present,0.0,-30.0,NaN,NaN,09:55:00,09:55:00,09:55:01,09:55:01,155,NaN,NaN,No,2.0.0,NaN,03/20/2018,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-382166...,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,09:54:45,09:54:48,MP3,alexh,multipatch patch seq pilot,NaN,SUCCESS,NaN,multi_1803

#### Resets index

In [11]:
#json_df = json_df.reset_index(drop = True)

In [12]:
display(json_df.head())

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,date,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID
0,2018-04-18,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,1.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:20:14 -07:00,13:20:13 -07:00,13:20:14 -07:00,NaN,PXS4_180418_551_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:19:59 -07:00,13:20:00 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
1,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,2.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_present,-80.0,-80.0,NaN,NaN,13:21:44 -07:00,13:21:44 -07:00,13:21:45 -07:00,NaN,PXS4_180418_552_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:21:26 -07:00,13:21:31 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
2,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,3.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:22:39 -07:00,13:22:38 -07:00,13:22:39 -07:00,NaN,PXS4_180418_553_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:22:25 -07:00,13:22:27 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
3,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,4.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,not_intentionally,nucleus_present,-80.0,-80.0,NaN,NaN,13:23:59 -07:00,13:23:58 -07:00,13:23:59 -07:00,NaN,PXS4_180418_554_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:23:04 -07:00,13:23:09 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
4,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,5.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_absent,-80.0,-80.0,NaN,NaN,13:25:01 -07:00,13:25:01 -07:00,13:25:01 -07:00,NaN,PXS4_180418_555_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:24:37 -07:00,13:24:40 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN


In [13]:
display(json_df.tail())

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,date,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID
0,11/01/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,10/12/2017,11/02/2017 12:03,NaN,NaN,NaN,1000,alexa fill visible post experiment.,Cell Swelled,no,nucleus_present,-100.0,-50.0,No Bubbles,NaN,12:31:55,12:27:45,12:35:06,12:31:55,053,NaN,NaN,Yes,2.0.0,NaN,10/05/2017,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai14-354918.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,12:18:43,12:20:40,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171030_03_B02_M
1,11/01/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 6a",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,10/12/2017,11/02/2017 12:03,NaN,NaN,NaN,1000,alexa fill visible post experiment.,NaN,no,nucleus_absent,-60.0,-25.0,No Bubbles,NaN,13:01:10,12:58:33,13:03:05,13:01:11,054,NaN,NaN,Yes,2.0.0,NaN,10/05/2017,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai14-354918.04.01,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,12:49:54,12:51:25,5,rustym,NaN,NaN,SUCCESS,Patch/Cell Unstable,EPhys_171030_03_B02_M
0,10/25/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,10/12/2017,10/26/2017 14:36,NaN,NaN,NaN,15,alexa fill visible post experiment. Forgot to...,Fluorescence in Pipette,intentionally,nucleus_present,-25.0,-25.0,No Bubbles,NaN,15:05:15,15:04:15,15:08:05,15:05:15,053,NaN,NaN,Yes,2.0.0,NaN,10/05/2017,NaN,NaN,NaN,NaN,NaN,Oxtr-T2A-Cre;Ai14-355298.03.01,NaN,NaN,NaN,NaN,NaN,NaN,6.2,NaN,14:53:08,14:55:01,4,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171023_03_B04_M
1,10/25/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,2.0,NaN,NaN,10/12/2017,10/26/2017 14:36,NaN,NaN,NaN,1000,alexa fill visible post experiment,Fluorescence in Pipette,no,nucleus_present,-25.0,-25.0,Small Bubbles,NaN,15:31:38,15:30:51,15:33:33,15:31:38,054,NaN,NaN,Yes,2.0.0,NaN,10/05/2017,NaN,NaN,NaN,NaN,NaN,Oxtr-T2A-Cre;Ai14-355298.03.01,NaN,NaN,NaN,NaN,NaN,NaN,4.1,NaN,15:21:46,15:23:15,4,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171023_03_B04_M
1,10/02/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 4",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,10/03/2017 14:35,NaN,NaN,NaN,25,Alexa fill not visible,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_absent,-25.0,-25.0,Small Bubbles,NaN,14:58:42,14:58:16,15:03:40,14:58:44,356,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Nr5a1-Cre;Ai14-350880.05.02,NaN,NaN,NaN,NaN,NaN,NaN,4.8,NaN,14:50:27,14:51:26,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_D04_M


In [14]:
#json_df = json_df.reset_index(drop=True)

In [15]:
#json_df['Date'] = json_df['extraction.tubeID'].str[5:11] #Not sure if available for all samples

In [16]:
#json_df = json_df.set_index('Date')

In [17]:
#json_df.sort_index(inplace = True)

##### String Slicing after Hours:Minutes:Seconds from RetractionEnd & WholeCell columns

In [18]:
#json_df1["extraction.timeRetractionEnd"] = json_df1["extraction.timeRetractionEnd"].str[:8]
#json_df1["recording.timeWholeCellStart"] = json_df1["recording.timeWholeCellStart"].str[:8]
#json_df1["extraction.timeEnd"] = json_df1["extraction.timeEnd"].str[:8]

In [19]:
#json_df1.info()

In [20]:
#json_df1["Patch Duration1"] = json_df1["extraction.timeEnd"] - json_df1["recording.timeWholeCellStart"]
#json_df1["Patch Duration2"] = json_df1["extraction.timeRetractionEnd"] - json_df1["recording.timeWholeCellStart"]
#display(json_df1.tail())

In [21]:
#Procdure for this script
#1) Create Patch Duration Column for that contains all json versions
#2) Merge 63x_df1 with json_df1
#3) Add in Post Patch Script 
#4) Graph Patch Duration vs Post Patch Outcome
#5) Graph Patch Duration vs Morphology(63x)
#6) Graph Patch Duration vs Morphology(63x) calls per post patch outcome

In [22]:
#***Problems***
#1) Different JSON versions cause problems for creating patch duration columns 
#   ExtEndTime - WholeCell = patch_dur1
#   RetEndTime - WholeCell = patch_dur2
#2) Different JSON versions cause problems for the post patch fields 
#   Nucleated...etc
#   nucleus_present OR nucleus_absent 

In [23]:
#Filtering columns in json_df 
#1) 'status' filter ONLY SUCCESS, SUCCESS(low confidence), SUCCESS(high confidence)
#    json_df = json_df[json_df['status'] == 'SUCCESS', 'SUCCESS(low confidence)', 'SUCCESS(high confidence)']
#2) 'approach.creCell' filter ONLY Cre+

#You can filter form version to find the differences between the jem forms 

In [24]:
#Necessary columns from json_df
# 'date', 
# 'recording.timeWholeCellStart', 
# 'extraction.timeEnd'
# 'extraction.timeRetractionEnd', 
# 'extraction.postPatch',
# 'extraction.nucleus',
# 'extraction.endPipetteR'